In [1]:
#import libraries
import os
import cv2
import numpy as np
import keras
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, GlobalMaxPooling2D, Activation, Multiply, Conv2D, Concatenate, Flatten
from keras.models import Model
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from numpy.random import seed
from keras.layers.core import Activation
from tensorflow.keras.utils import get_custom_objects
from keras import backend as K

seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
tf.config.experimental.enable_op_determinism()

2023-02-20 08:29:33.675029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 08:29:39.918235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cserv1_a/soc_ug/fy19iars/project/lib/python3.9/site-packages/cv2/../../lib64:/uollinapps/AppsData/src/vscode/1.71.2-1663191299.el7/lib64
2023-02-20 08:29:39.918273: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-20 08:29:55.112013: W tensorflow/compiler/xla/stream_executor/platform/defa

In [2]:
def list_labels(file):
    labels_file = open(file, "r")
    labels = []
    
    for line in labels_file:
        label = line.strip()
        labels.append(label)
    
    labels_file.close()
    
    return labels


#list of all labels
class_names = list_labels("./CamSDD/Labels.txt")
class_name_labels = {class_name:i for i, class_name in enumerate(class_names)}



def load_data(folder):
    Category = ["training", "test", "validation"]
    output = []
    
    for category in Category:
        print("Loading {}".format(category))
        path = os.path.join(folder, category)
        print(path)
        images = []
        labels = []
        
        for sub_folder in os.listdir(path):
            label = class_name_labels[sub_folder]
            
            #Iterating through all images
            for file in os.listdir(os.path.join(path, sub_folder)):
                
                #getting the image path
                img_path = os.path.join(os.path.join(path, sub_folder), file)
                
                #appending image and corresponding label
                images.append(cv2.resize(cv2.imread(img_path), (224, 224)))
                # images.append(cv2.imread(img_path))
                labels.append(label)
            
        #check that data type doesn't affect accuracy
        images = (np.array(images, dtype='float32')/127.5) - 1
        labels = np.array(labels, dtype='int8')
        
        output.append((images, labels))
        
    return output

#channel module
def channel_attention_module(x, ratio, bias):
    b, _, _, channel = x.shape
    # MLP shared layer
    l1 = Dense(channel//ratio, activation="relu", use_bias=bias)
    l2 = Dense(channel, use_bias=bias)
    
    # Global Average pooling
    x1 = GlobalAveragePooling2D()(x)
    x1 = l1(x1)
    x1 = l2(x1)
    
    # Global Max pooling
    x2 = GlobalMaxPooling2D()(x)
    x2 = l1(x2)
    x2 = l2(x2)
    
    # Adding both and applying sigmoid
    features = x1 + x2
    features = Activation("sigmoid")(features)
    features = Multiply()([x, features])
    
    return features

# spatial attention module
def spatial_attention_module(x):
    # Average pooling
    x1 = tf.reduce_mean(x, axis=-1)
    x1 = tf.expand_dims(x1, axis=-1)
    
    # Max pooling
    x2 = tf.reduce_max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    
    # Concatenate
    features = Concatenate()([x1, x2])
    
    # Conv layer
    features = Conv2D(1, kernel_size=7, padding="same", activation="sigmoid")(features)
    features = Multiply()([x, features])
    
    return features
    
# CBAM
def CBAM(x, ratio, bias):
    x = channel_attention_module(x, ratio=ratio, bias=bias)
    x = spatial_attention_module(x)
    return x

def elu2(x):
    return K.switch(K.less_equal(x, 1), K.exp(x) - 1, x)

In [3]:
#shuffling train data
(train_images, train_labels), (test_images, test_labels), (validation_images, validation_labels)= load_data("./CamSDD")
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
validation_images, validation_labels = shuffle(validation_images, validation_labels, random_state=25)

Loading training
./CamSDD/training
Loading test
./CamSDD/test
Loading validation
./CamSDD/validation


In [4]:
base_model=MobileNet(weights='imagenet', include_top=False) 

x=base_model.output
x=CBAM(x, 4, True)
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='sigmoid')(x)
x=Dropout(0.7)(x)
output = Dense(30, activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

for layer in base_model.layers:
    layer.trainable = False

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history = model.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor], shuffle=False)

2023-02-20 06:11:51.709560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cserv1_a/soc_ug/fy19iars/project/lib/python3.9/site-packages/cv2/../../lib64:/uollinapps/AppsData/src/vscode/1.71.2-1663191299.el7/lib64
2023-02-20 06:11:51.709636: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-20 06:11:51.709709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feng-linux-08.leeds.ac.uk): /proc/driver/nvidia/version does not exist
2023-02-20 06:11:51.711705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:

Epoch 1/100
495/495 [==============================] - 82s 157ms/step - loss: 2.3061 - accuracy: 0.4100 - val_loss: 0.9837 - val_accuracy: 0.8450
Epoch 2/100
495/495 [==============================] - 78s 158ms/step - loss: 0.9146 - accuracy: 0.7693 - val_loss: 0.5912 - val_accuracy: 0.8750
Epoch 3/100
495/495 [==============================] - 78s 157ms/step - loss: 0.6425 - accuracy: 0.8293 - val_loss: 0.4702 - val_accuracy: 0.8867
Epoch 4/100
495/495 [==============================] - 76s 154ms/step - loss: 0.5034 - accuracy: 0.8612 - val_loss: 0.3992 - val_accuracy: 0.9017
Epoch 5/100
495/495 [==============================] - 77s 156ms/step - loss: 0.4266 - accuracy: 0.8819 - val_loss: 0.3583 - val_accuracy: 0.9067
Epoch 6/100
495/495 [==============================] - 77s 155ms/step - loss: 0.3581 - accuracy: 0.8984 - val_loss: 0.3290 - val_accuracy: 0.9117
Epoch 7/100
495/495 [==============================] - 76s 154ms/step - loss: 0.3087 - accuracy: 0.9137 - val_loss: 0.3134 -

In [5]:
model.save("CBAM_r4_T")
score = model.evaluate(test_images, test_labels)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

INFO:tensorflow:Assets written to: CBAM_r4_T/assets


INFO:tensorflow:Assets written to: CBAM_r4_T/assets


19/19 [==============================] - 5s 216ms/step - loss: 0.2395 - accuracy: 0.9233
Test loss: 0.23949870467185974
Test accuracy: 0.9233333468437195


In [6]:
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
base_model2=MobileNet(weights='imagenet', include_top=False) 
x2=base_model2.output
x2=CBAM(x2, 4, False)
x2=GlobalAveragePooling2D()(x2)
x2=Dense(1024, activation='sigmoid')(x2)
x2=Dropout(0.7)(x2)
output2 = Dense(30, activation="softmax")(x2)

model2=Model(inputs=base_model2.input,outputs=output2)

for layer in base_model2.layers:
    layer.trainable = False

monitor2 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history2 = model2.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor2], shuffle=False)

Epoch 1/100
495/495 [==============================] - 79s 155ms/step - loss: 2.3059 - accuracy: 0.4096 - val_loss: 0.9835 - val_accuracy: 0.8417
Epoch 2/100
495/495 [==============================] - 77s 156ms/step - loss: 0.9148 - accuracy: 0.7682 - val_loss: 0.5918 - val_accuracy: 0.8750
Epoch 3/100
495/495 [==============================] - 77s 155ms/step - loss: 0.6428 - accuracy: 0.8299 - val_loss: 0.4708 - val_accuracy: 0.8883
Epoch 4/100
495/495 [==============================] - 77s 155ms/step - loss: 0.5034 - accuracy: 0.8617 - val_loss: 0.3998 - val_accuracy: 0.9000
Epoch 5/100
495/495 [==============================] - 77s 155ms/step - loss: 0.4263 - accuracy: 0.8813 - val_loss: 0.3587 - val_accuracy: 0.9083
Epoch 6/100
495/495 [==============================] - 77s 155ms/step - loss: 0.3576 - accuracy: 0.8984 - val_loss: 0.3298 - val_accuracy: 0.9100
Epoch 7/100
495/495 [==============================] - 76s 154ms/step - loss: 0.3082 - accuracy: 0.9142 - val_loss: 0.3141 -

In [7]:
model2.save("CBAM_r4_F")
score2 = model2.evaluate(test_images, test_labels)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

INFO:tensorflow:Assets written to: CBAM_r4_F/assets


INFO:tensorflow:Assets written to: CBAM_r4_F/assets


19/19 [==============================] - 5s 210ms/step - loss: 0.2429 - accuracy: 0.9250
Test loss: 0.24288415908813477
Test accuracy: 0.925000011920929


In [8]:
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
base_model3=MobileNet(weights='imagenet', include_top=False) 
x3=base_model3.output
x3=CBAM(x3, 16, True)
x3=GlobalAveragePooling2D()(x3)
x3=Dense(1024, activation='sigmoid')(x3)
x3=Dropout(0.7)(x3)
output3 = Dense(30, activation="softmax")(x3)

model3=Model(inputs=base_model3.input,outputs=output3)

for layer in base_model3.layers:
    layer.trainable = False

monitor3 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model3.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history3 = model3.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor3], shuffle=False)

Epoch 1/100
495/495 [==============================] - 78s 154ms/step - loss: 2.3285 - accuracy: 0.4040 - val_loss: 0.9717 - val_accuracy: 0.8367
Epoch 2/100
495/495 [==============================] - 75s 152ms/step - loss: 0.9097 - accuracy: 0.7691 - val_loss: 0.5888 - val_accuracy: 0.8783
Epoch 3/100
495/495 [==============================] - 75s 151ms/step - loss: 0.6519 - accuracy: 0.8244 - val_loss: 0.4740 - val_accuracy: 0.8917
Epoch 4/100
495/495 [==============================] - 75s 151ms/step - loss: 0.5226 - accuracy: 0.8542 - val_loss: 0.4056 - val_accuracy: 0.9000
Epoch 5/100
495/495 [==============================] - 75s 151ms/step - loss: 0.4512 - accuracy: 0.8726 - val_loss: 0.3668 - val_accuracy: 0.9067
Epoch 6/100
495/495 [==============================] - 75s 151ms/step - loss: 0.3912 - accuracy: 0.8899 - val_loss: 0.3399 - val_accuracy: 0.9083
Epoch 7/100
495/495 [==============================] - 76s 153ms/step - loss: 0.3471 - accuracy: 0.9008 - val_loss: 0.3237 -

In [9]:
model3.save("CBAM_r16_T")
score3 = model3.evaluate(test_images, test_labels)
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

INFO:tensorflow:Assets written to: CBAM_r16_T/assets


INFO:tensorflow:Assets written to: CBAM_r16_T/assets


19/19 [==============================] - 5s 219ms/step - loss: 0.2361 - accuracy: 0.9317
Test loss: 0.23611727356910706
Test accuracy: 0.9316666722297668


In [4]:
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
base_model4=MobileNet(weights='imagenet', include_top=False) 
x4=base_model4.output
x4=CBAM(x4, 16, False)
x4=GlobalAveragePooling2D()(x4)
x4=Dense(1024, activation='sigmoid')(x4)
x4=Dropout(0.7)(x4)
output4 = Dense(30, activation="softmax")(x4)

model4=Model(inputs=base_model4.input,outputs=output4)

for layer in base_model4.layers:
    layer.trainable = False

monitor4 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model4.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history4 = model4.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor4], shuffle=False)

2023-02-20 08:32:20.802728: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cserv1_a/soc_ug/fy19iars/project/lib/python3.9/site-packages/cv2/../../lib64:/uollinapps/AppsData/src/vscode/1.71.2-1663191299.el7/lib64
2023-02-20 08:32:20.802767: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-20 08:32:20.802807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (feng-linux-03.leeds.ac.uk): /proc/driver/nvidia/version does not exist
2023-02-20 08:32:20.804034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:

Epoch 1/100
495/495 [==============================] - 57s 110ms/step - loss: 2.3287 - accuracy: 0.4042 - val_loss: 0.9710 - val_accuracy: 0.8367
Epoch 2/100
495/495 [==============================] - 54s 109ms/step - loss: 0.9091 - accuracy: 0.7690 - val_loss: 0.5883 - val_accuracy: 0.8783
Epoch 3/100
495/495 [==============================] - 53s 107ms/step - loss: 0.6518 - accuracy: 0.8241 - val_loss: 0.4740 - val_accuracy: 0.8917
Epoch 4/100
495/495 [==============================] - 53s 106ms/step - loss: 0.5224 - accuracy: 0.8538 - val_loss: 0.4057 - val_accuracy: 0.9000
Epoch 5/100
495/495 [==============================] - 53s 107ms/step - loss: 0.4511 - accuracy: 0.8731 - val_loss: 0.3669 - val_accuracy: 0.9083
Epoch 6/100
495/495 [==============================] - 52s 106ms/step - loss: 0.3911 - accuracy: 0.8907 - val_loss: 0.3398 - val_accuracy: 0.9067
Epoch 7/100
495/495 [==============================] - 52s 106ms/step - loss: 0.3470 - accuracy: 0.9010 - val_loss: 0.3235 -

In [5]:
model4.save("CBAM_r16_F")
score4 = model4.evaluate(test_images, test_labels)
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])

INFO:tensorflow:Assets written to: CBAM_r16_F/assets


INFO:tensorflow:Assets written to: CBAM_r16_F/assets


19/19 [==============================] - 4s 148ms/step - loss: 0.2356 - accuracy: 0.9300
Test loss: 0.23561060428619385
Test accuracy: 0.9300000071525574


In [6]:
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
base_model5=MobileNet(weights='imagenet', include_top=False) 
x5=base_model5.output
x5=CBAM(x5, 8, True)
x5=GlobalAveragePooling2D()(x5)
x5=Dense(1024, activation='sigmoid')(x5)
x5=Dropout(0.7)(x5)
output5 = Dense(30, activation="softmax")(x5)

model5=Model(inputs=base_model5.input,outputs=output5)

for layer in base_model5.layers:
    layer.trainable = False

monitor5 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model5.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history5 = model5.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor5], shuffle=False)

Epoch 1/100
495/495 [==============================] - 56s 109ms/step - loss: 2.3389 - accuracy: 0.4010 - val_loss: 0.9932 - val_accuracy: 0.8400
Epoch 2/100
495/495 [==============================] - 53s 108ms/step - loss: 0.9212 - accuracy: 0.7676 - val_loss: 0.5952 - val_accuracy: 0.8750
Epoch 3/100
495/495 [==============================] - 53s 107ms/step - loss: 0.6533 - accuracy: 0.8259 - val_loss: 0.4759 - val_accuracy: 0.8883
Epoch 4/100
495/495 [==============================] - 53s 107ms/step - loss: 0.5185 - accuracy: 0.8562 - val_loss: 0.4058 - val_accuracy: 0.9000
Epoch 5/100
495/495 [==============================] - 53s 108ms/step - loss: 0.4449 - accuracy: 0.8734 - val_loss: 0.3637 - val_accuracy: 0.9067
Epoch 6/100
495/495 [==============================] - 53s 107ms/step - loss: 0.3820 - accuracy: 0.8940 - val_loss: 0.3354 - val_accuracy: 0.9117
Epoch 7/100
495/495 [==============================] - 53s 107ms/step - loss: 0.3370 - accuracy: 0.9038 - val_loss: 0.3191 -

In [7]:
model5.save("CBAM_r8_T")
score5 = model5.evaluate(test_images, test_labels)
print('Test loss:', score5[0])
print('Test accuracy:', score5[1])

INFO:tensorflow:Assets written to: CBAM_r8_T/assets


INFO:tensorflow:Assets written to: CBAM_r8_T/assets


19/19 [==============================] - 4s 150ms/step - loss: 0.2365 - accuracy: 0.9233
Test loss: 0.23650220036506653
Test accuracy: 0.9233333468437195


In [8]:
seed(25)
tf.random.set_seed(25)
tf.keras.utils.set_random_seed(25)
base_model6=MobileNet(weights='imagenet', include_top=False) 
x6=base_model6.output
x6=CBAM(x6, 8, False)
x6=GlobalAveragePooling2D()(x6)
x6=Dense(1024, activation='sigmoid')(x6)
x6=Dropout(0.7)(x6)
output6 = Dense(30, activation="softmax")(x6)

model6=Model(inputs=base_model6.input,outputs=output6)

for layer in base_model6.layers:
    layer.trainable = False

monitor6 = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, restore_best_weights=True)
model6.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=["accuracy"])
history6 = model6.fit(train_images, train_labels, batch_size=20, epochs=100, validation_data=(validation_images,validation_labels), callbacks=[monitor6], shuffle=False)

Epoch 1/100
495/495 [==============================] - 56s 110ms/step - loss: 2.3386 - accuracy: 0.4004 - val_loss: 0.9928 - val_accuracy: 0.8400
Epoch 2/100
495/495 [==============================] - 53s 106ms/step - loss: 0.9213 - accuracy: 0.7678 - val_loss: 0.5952 - val_accuracy: 0.8733
Epoch 3/100
495/495 [==============================] - 53s 106ms/step - loss: 0.6526 - accuracy: 0.8260 - val_loss: 0.4753 - val_accuracy: 0.8900
Epoch 4/100
495/495 [==============================] - 53s 107ms/step - loss: 0.5182 - accuracy: 0.8558 - val_loss: 0.4058 - val_accuracy: 0.8983
Epoch 5/100
495/495 [==============================] - 53s 107ms/step - loss: 0.4447 - accuracy: 0.8734 - val_loss: 0.3633 - val_accuracy: 0.9050
Epoch 6/100
495/495 [==============================] - 53s 106ms/step - loss: 0.3823 - accuracy: 0.8936 - val_loss: 0.3350 - val_accuracy: 0.9133
Epoch 7/100
495/495 [==============================] - 53s 107ms/step - loss: 0.3369 - accuracy: 0.9040 - val_loss: 0.3181 -

In [9]:
model6.save("CBAM_r8_F")
score6 = model6.evaluate(test_images, test_labels)
print('Test loss:', score6[0])
print('Test accuracy:', score6[1])

INFO:tensorflow:Assets written to: CBAM_r8_F/assets


INFO:tensorflow:Assets written to: CBAM_r8_F/assets


19/19 [==============================] - 4s 147ms/step - loss: 0.2472 - accuracy: 0.9183
Test loss: 0.24724383652210236
Test accuracy: 0.9183333516120911
